In [1]:
import pandas as pd
import os 
import allel
import random
import subprocess
import pickle
from os import path as op
import numpy as np
from dask import delayed
import dask.dataframe as dd



path = '../../grene/'
flowers = pd.read_csv(path + 'data/samples_data.csv')

#path_meixi = '/carnegie/data/Shared/Labs/Moi/Everyone/meixilin'

flowers = pd.read_csv(path + '/data/merged_samples_data.csv')

num_flowers_map = flowers.set_index('sample_name')['total_flower_counts'].to_dict()

path_all_af_indexed = '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/baypass_terminal/merged_hapFIRE_allele_frequency_indexed.csv'

#Choose SNPs for GEA structure correction in baypass

#os.listdir('/home/tbellagio/safedata/meixilin/grenenet/metadata/data')

#pd.read_csv('/home/tbellagio/safedata/ath_evo/grenephase1-data/merged_tables/merged_hapFIRE_allele_frequency_LDpruned.txt', sep = '\t')

#all_samples_af = os.listdir('/carnegie/nobackup/scratch/xwu/grenet/hapFIRE_frequencies/samples/snp_frequency/')

## the size of this file is 3235481 * 745  
path_all_af = '/carnegie/nobackup/scratch/xwu/grenet/merged_frequency/merged_hapFIRE_allele_frequency.csv'

## path to ld prunned allele freq
path_ldp_af = '/carnegie/nobackup/scratch/xwu/grenet/merged_frequency/merged_hapFIRE_allele_frequency_LDpruned.txt'

final_gen = pd.read_csv('../final_gen.csv')['sample_name'].to_list()
#final_gen.append('0')

/home/tbellagio/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
gfile_ddf = dd.read_csv(path_all_af_indexed, usecols=final_gen)

gfile_ddf = gfile_ddf.compute()

gfile_ddf

allele_counts = {}
for column in gfile_ddf.columns:
    num_flowers = num_flowers_map.get(column, 0)  # Default to 0 if not found
    allele_counts[column + '_minor'] = gfile_ddf[column] * num_flowers
    allele_counts[column + '_major'] = (1 - gfile_ddf[column]) * num_flowers

gfile_ddf = None

result_df = pd.DataFrame(allele_counts).round().astype(int)

allele_counts = None


,1_2_2_minor,1_2_2_major,1_2_3_minor,1_2_3_major,1_3_1_minor,1_3_1_major,1_3_4_minor,1_3_4_major,1_3_5_minor,1_3_5_major,...,60_1_8_minor,60_1_8_major,60_1_9_minor,60_1_9_major,60_1_10_minor,60_1_10_major,60_1_11_minor,60_1_11_major,60_1_12_minor,60_1_12_major
0,2,192,9,108,0,24,1,19,6,66,...,1,23,0,7,3,18,4,16,2,18
1,66,128,38,79,18,6,12,8,25,47,...,6,18,5,2,5,16,10,10,12,8
2,0,194,0,117,0,24,0,20,0,72,...,0,24,0,7,0,21,0,20,0,20
3,22,172,4,113,3,21,0,20,0,72,...,0,24,0,7,0,21,0,20,0,20
4,10,184,2,115,0,24,0,20,13,59,...,3,21,0,7,0,21,2,18,0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6063,2,192,4,113,0,24,0,20,0,72,...,0,24,0,7,0,21,0,20,0,20
6064,1,193,0,117,0,24,0,20,0,72,...,0,24,0,7,0,21,0,20,0,20
6065,0,194,0,117,0,24,0,20,0,72,...,0,24,0,7,0,21,0,20,0,20
6066,97,97,46,71,19,5,7,13,48,24,...,18,6,2,5,8,13,11,9,7,13


In [6]:
final_gen = pd.read_csv('../final_gen.csv')['sample_name'].to_list()

In [14]:
cols_final_gen = [col for col in result_df.columns if any(element in col for element in final_gen)]

In [15]:
result_df = result_df[cols_final_gen]

In [17]:
dict_snps = '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/var_pos_grenenet.csv'

dict_snps = pd.read_csv(dict_snps)

In [19]:
dict_snps['id']

0               1_291
1               1_346
2               1_349
3               1_352
4               1_353
              ...    
3235475    5_26975208
3235476    5_26975215
3235477    5_26975267
3235478    5_26975272
3235479    5_26975288
Name: id, Length: 3235480, dtype: object

In [20]:
result_df.index = dict_snps['id']

In [21]:
result_dfdd = result_df.drop_duplicates()

In [22]:
result_dfdd.to_csv('baypass_allele_counts_drop_duplicates_terminal_gen.csv')

,1_2_2_minor,1_2_2_major,1_2_3_minor,1_2_3_major,1_3_1_minor,1_3_1_major,1_3_4_minor,1_3_4_major,1_3_5_minor,1_3_5_major,...,60_1_8_minor,60_1_8_major,60_1_9_minor,60_1_9_major,60_1_10_minor,60_1_10_major,60_1_11_minor,60_1_11_major,60_1_12_minor,60_1_12_major
id,,,,,,,,,,,,,,,,,,,,,
1_291,2,192,9,108,0,24,1,19,6,66,...,1,23,0,7,3,18,4,16,2,18
1_346,66,128,38,79,18,6,12,8,25,47,...,6,18,5,2,5,16,10,10,12,8
1_349,0,194,0,117,0,24,0,20,0,72,...,0,24,0,7,0,21,0,20,0,20
1_352,22,172,4,113,3,21,0,20,0,72,...,0,24,0,7,0,21,0,20,0,20
1_353,10,184,2,115,0,24,0,20,13,59,...,3,21,0,7,0,21,2,18,0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5_26975065,2,192,1,116,0,24,0,20,1,71,...,0,24,0,7,0,21,0,20,0,20
5_26975121,26,168,3,114,3,21,0,20,17,55,...,1,23,0,7,2,19,2,18,0,20
5_26975144,1,193,1,116,0,24,2,18,0,72,...,0,24,0,7,3,18,1,19,1,19


In [23]:
result_dfdd.to_csv('baypass_allele_counts_drop_duplicates_terminal_gen.csv')

In [5]:
minor_col = [i for i in result_df.columns if 'minor' in i]

In [7]:
result_df = result_df[minor_col]

In [11]:
result_df = result_df.reset_index(drop=True)

In [15]:
index_to_add = pd.read_csv('index_to_add.csv')

In [17]:
index_to_add['0']

0               1_291
1               1_346
2               1_349
3               1_352
4               1_353
              ...    
3235475    5_26975208
3235476    5_26975215
3235477    5_26975267
3235478    5_26975272
3235479    5_26975288
Name: 0, Length: 3235480, dtype: object

In [18]:
result_df.index = index_to_add['0']

In [21]:
alt_al = result_df.sum(axis=1)

In [22]:
alt_al.to_csv('total_number_of_alt_alleles.csv')

In [30]:
alt_al = pd.read_csv('total_number_of_alt_alleles.csv').set_index('0')

In [32]:
alt_al.columns = ['alt_al']

In [35]:
alt_al[alt_al['alt_al'] == 0]

,alt_al
0,
1_24037101,0
1_24044289,0
1_24059743,0
1_24060445,0
1_24074620,0
...,...
5_25306573,0
5_25307390,0
5_25307958,0
